## Multilingual Code-switching Identification via LSTM Recurrent Neural Networks

This notebook implements the model described in the paper <a href="http://www.aclweb.org/anthology/W/W16/W16-58.pdf#page=62"> Multilingual Code-switching Identification via LSTM Recurrent Neural Networks.</a>

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import csv
from keras.preprocessing import sequence
from itertools import chain
from keras.utils import np_utils

Using Theano backend.


### Data Preparation

In [2]:
# read the training data in tsv format
train_df=pd.read_csv('data/train.tsv',sep='\t',quoting=csv.QUOTE_NONE,names=['tweet_id','user_id','start','end','token','label'])

In [3]:
# read the dev data in tsv format
dev_df=pd.read_csv('data/dev.tsv',sep='\t',quoting=csv.QUOTE_NONE,names=['tweet_id','user_id','start','end','token','label'])

In [4]:
train_df.head()

,tweet_id,user_id,start,end,token,label
0,426559582895230976,169403434,0,6,Bambino,fw
1,426559582895230976,169403434,8,9,:D,other
2,416002744642576385,169403434,0,12,@JhonCristian,other
3,416002744642576385,169403434,14,14,♡,other
4,416002744642576385,169403434,16,18,<<<,other


In [5]:
dev_df.head()

,tweet_id,user_id,start,end,token,label
0,440338729215860736,21327323,0,4,11:11,other
1,440338729215860736,21327323,5,9,.....,other
2,440338729215860736,21327323,10,13,make,lang1
3,440338729215860736,21327323,15,15,a,lang1
4,440338729215860736,21327323,17,20,wish,lang1


In [6]:
#group data by tweet_id

train_grouped=train_df.groupby('tweet_id',as_index=False, sort=False)
train_tweets=train_grouped['token'].apply(list).values
train_labels=train_grouped['label'].apply(list).values

In [7]:
train_tweets 

array([['Bambino', ':D'], ['@JhonCristian', '♡', '<<<'],
       ['@Audreyy_torres', 'mhmmm', '-.-'], ...,
       ['ABOTABOTABOTABOOOOOO', '!!!!!!', 'PIPIPPIPI'],
       ['@ThatGirlYahs', 'callata', 'borrega', 'pelos', 'necios', '!!!!!'],
       ['id', 'omo', 'una', 'plaga', 'contra', 'el', 'aburrimiento', 'del', 'mundo']], dtype=object)

In [8]:
train_labels

array([['fw', 'other'], ['other', 'other', 'other'],
       ['other', 'unk', 'other'], ..., ['lang2', 'other', 'unk'],
       ['other', 'lang2', 'lang2', 'lang2', 'lang2', 'other'],
       ['lang2', 'lang2', 'lang2', 'lang2', 'lang2', 'lang2', 'lang2', 'lang2', 'lang2']], dtype=object)

In [9]:
#repeat same for dev data
dev_grouped=dev_df.groupby('tweet_id',as_index=False, sort=False)
dev_tweets=dev_grouped['token'].apply(list).values
dev_labels=dev_grouped['label'].apply(list).values

In [10]:
dev_tweets

array([['11:11', '.....', 'make', 'a', 'wish', '.......', 'night', 'night'],
       ['@YiseBabee', 'i', 'wish', 'i', 'could', 'be', 'ignorant'],
       ['Bout', 'to', 'get', 'my', 'Fat', 'ON'], ...,
       ['Oomf', ',', 'dm', 'me', '😉'],
       ['Some', 'drunk', 'guy', 'just', 'asked', 'what', 'time', 'it', 'was', 'and', 'i', 'told', 'him', '10:30', 'lmao', '!'],
       ['This', 'person', 'just', 'got', 'pulled', 'over', 'on', 'a', 'bicycle', 'with', 'a', 'motor', '😂😂']], dtype=object)

In [11]:
dev_labels

array([['other', 'other', 'lang1', 'lang1', 'lang1', 'other', 'lang1', 'lang1'],
       ['other', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1'],
       ['lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1'], ...,
       ['lang1', 'other', 'lang1', 'lang1', 'other'],
       ['lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'other', 'lang1', 'other'],
       ['lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'lang1', 'other']], dtype=object)

### Building vocabulary

Here, we will look at the training data to build the vocabulary, word to index, character to index and tags to index. Also, we create reverse mapping from index to word, index to character and index to tags.

In [12]:
# REf: https://github.com/neubig/yrsnlp-2016/blob/master/bow.ipynb
# w2i : word to index [assign unique ids for words in training data]
# c2i : character to index [assign unique ids for characters  in training data]
# t2i : tag to index [assign unique ids for tags in training data]
# reserve 0 for padding and 1 for unknown words


w2i = defaultdict(lambda: len(w2i))
c2i = defaultdict(lambda: len(c2i))
t2i = defaultdict(lambda: len(t2i))
PAD=w2i['<padding>']
UNK = w2i["<unk>"]
CHAR_PAD=c2i['<padding>']
CHAR_UNK =c2i["<unk>"]

In [13]:
from pprint import pprint
pprint (w2i)

defaultdict(<function <lambda> at 0x11d053488>, {'<padding>': 0, '<unk>': 1})


In [14]:
#iterate over the tokens in training data to assign unique ids for tokens and characters

for tweet in train_tweets:
    for token in tweet:
        w2i[token]
        for c in token:
            c2i[c]
w2i=defaultdict(lambda : UNK,w2i) #prevents assigning new ids for new words, instead they are assigned the unk token id
c2i=defaultdict(lambda : CHAR_UNK,c2i)
i2w = {v: k for k, v in w2i.items()}
i2c= {v: k for k, v in c2i.items()}
n_words=len(w2i)
n_chars=len(c2i)

vocabulary=frozenset([k for k,v in w2i.items()])

MAX_LEN=max([len(word) for word in vocabulary]) #longest token length

In [15]:
for tweet_label in train_labels:
    for label in tweet_label:
        t2i[label]

i2t={v: k for k, v in t2i.items()}
tags=frozenset([k for k,v in t2i.items()])
n_tags=len(t2i)

In [16]:
print (MAX_LEN)

50


In [17]:
print (len(vocabulary))

17655


In [18]:
print (n_words)

17655


In [19]:
print (tags)

frozenset({'lang2', 'fw', 'ambiguous', 'lang1', 'unk', 'mixed', 'other', 'ne'})


In [20]:
print (len(tags))

8


In [21]:
print (n_tags)

8


In [22]:
t2i

defaultdict(<function __main__.<lambda>>,
            {'ambiguous': 6,
             'fw': 0,
             'lang1': 3,
             'lang2': 4,
             'mixed': 7,
             'ne': 5,
             'other': 1,
             'unk': 2})

### Converting tweets  tokens and labels to their ids

Now, we will transform the tweet tokens and tags to their corresponding ids. 

In [23]:
# replace tokens and labels with their ids

transform_train_tweet= [[w2i[token] for token in tweet] for tweet in train_tweets]
transform_train_labels=[[t2i[label] for label in tweet_label] for tweet_label in train_labels]

In [24]:
#first five tweets
transform_train_tweet[:5]

[[2, 3], [4, 5, 6], [7, 8, 9], [10, 11], [10]]

In [25]:
#last five tweets
transform_train_tweet[-5:]

[[1350, 3937],
 [17647, 256, 17648, 256, 17647, 256],
 [17649, 1366, 17650],
 [9026, 17651, 9997, 9358, 9998, 1610],
 [5055, 17652, 1293, 17653, 8625, 1354, 17654, 1303, 1760]]

In [26]:
transform_train_labels[:5]

[[0, 1], [1, 1, 1], [1, 2, 1], [2, 1], [2]]

In [27]:
transform_train_labels[-5:]

[[4, 1],
 [4, 1, 4, 1, 4, 1],
 [4, 1, 2],
 [1, 4, 4, 4, 4, 1],
 [4, 4, 4, 4, 4, 4, 4, 4, 4]]

In [28]:
#repeat same for dev dataset

transform_dev_tweet= [[w2i[token] for token in tweet] for tweet in dev_tweets]
transform_dev_labels=[[t2i[label] for label in tweet_label] for tweet_label in dev_labels]


In [29]:
transform_dev_tweet[-5:]

[[5411, 114, 1],
 [478, 177, 479],
 [5411, 182, 6128, 304, 32],
 [1039,
  5488,
  462,
  197,
  3575,
  67,
  423,
  72,
  55,
  268,
  880,
  3517,
  280,
  13878,
  241,
  48],
 [309, 119, 197, 177, 11807, 504, 116, 114, 1, 630, 114, 1, 1476]]

In [30]:
transform_dev_labels[-5:]

[[3, 3, 3],
 [3, 3, 3],
 [3, 1, 3, 3, 1],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1]]

In [31]:
#last dev tweet
dev_tweets[-1]

['This',
 'person',
 'just',
 'got',
 'pulled',
 'over',
 'on',
 'a',
 'bicycle',
 'with',
 'a',
 'motor',
 '😂😂']

In [32]:
#last dev tweets ids
transform_dev_tweet[-1]

[309, 119, 197, 177, 11807, 504, 116, 114, 1, 630, 114, 1, 1476]

In [33]:
print('bicycle' in vocabulary)

False


In [34]:
print('motor' in vocabulary)

False


In [35]:
w2i['motor']

1

In [36]:
w2i['bicycle']

1

Since "motor" and "bicycle" are not in the vocabulary, they are assigned the id for unk token

#### Character

We represent each training instance by sequnece of characters in a token.

In [37]:
tranform_train_tweet_chars=[[[c2i[c]  for c in token ] for token in tweet] for tweet in train_tweets]

In [38]:
tranform_train_tweet_chars[:5]

[[[2, 3, 4, 5, 6, 7, 8], [9, 10]],
 [[11, 12, 13, 8, 7, 14, 15, 6, 16, 17, 6, 3, 7], [18], [19, 19, 19]],
 [[11, 20, 21, 22, 15, 23, 24, 24, 25, 17, 8, 15, 15, 23, 16],
  [4, 13, 4, 4, 4],
  [26, 27, 26]],
 [[28, 4, 4, 4], [27, 27, 27]],
 [[28, 4, 4, 4]]]

In [39]:
tranform_dev_tweet_chars=[[[c2i[c]  for c in token ] for token in tweet] for tweet in dev_tweets]

In [40]:
tranform_dev_tweet_chars[:5]

[[[32, 32, 9, 32, 32],
  [27, 27, 27, 27, 27],
  [4, 3, 36, 23],
  [3],
  [38, 6, 16, 13],
  [27, 27, 27, 27, 27, 27, 27],
  [7, 6, 48, 13, 17],
  [7, 6, 48, 13, 17]],
 [[11, 41, 6, 16, 23, 2, 3, 5, 23, 23],
  [6],
  [38, 6, 16, 13],
  [6],
  [49, 8, 21, 31, 22],
  [5, 23],
  [6, 48, 7, 8, 15, 3, 7, 17]],
 [[2, 8, 21, 17], [17, 8], [48, 23, 17], [4, 24], [60, 3, 17], [61, 65]],
 [[29, 23],
  [23, 7, 49, 3, 7, 17, 3, 15, 6, 3],
  [81, 21, 23],
  [4, 23],
  [30, 6, 23, 15, 3, 16],
  [80],
  [22, 23],
  [31, 3],
  [4, 3, 7, 23, 15, 3],
  [81, 21, 23],
  [24, 8],
  [17, 23],
  [30, 23, 8],
  [64, 61, 7, 23, 10, 3, 24]],
 [[11, 14, 3, 31, 30, 3, 15, 24, 20, 15, 31, 14, 14],
  [10, 8],
  [24, 8, 21],
  [48, 21, 24, 16],
  [13, 3, 30, 23],
  [3],
  [16, 39, 3, 7, 6, 16, 13],
  [22, 23, 39, 3, 15, 17, 4, 23, 7, 17],
  [8, 15],
  [16, 23, 15, 30, 6, 49, 23, 16],
  [40, 40, 40]]]

### Building training and test data

For word model, we will represent each tweet token by its context. The contexts are some tokens to the right and left.

In [41]:
#REF: http://deeplearning.net/tutorial/rnnslu.html#rnnslu
def contextwin(l, win):
    '''
    win :: int corresponding to the size of the window
    given a list of indexes composing a sentence

    l :: array containing the word indexes

    it will return a list of list of indexes corresponding
    to context windows surrounding each word in the sentence
    '''
    assert (win % 2) == 1
    assert win >= 1
    l = list(l)

    lpadded = win // 2 * [PAD] + l + win // 2 * [PAD]
    out = [lpadded[i:(i + win)] for i in range(len(l))]

    assert len(out) == len(l)
    return out

In [42]:
context_window=5 #each training instance will have sequence length of 5. The token will be in the middle surrounded by the  right and left tokens.

#### Example transformed tweet

In [43]:
#0 indicates padding 
#pprint(" ".join([i2w[token] for token in transform_train_tweet[10]]))
pprint(transform_train_tweet[10])

[18, 13, 21, 11, 9]


In [44]:
#represent above tweet  tokens with context window of 5
pprint(contextwin(transform_train_tweet[10],context_window))

[[0, 0, 18, 13, 21],
 [0, 18, 13, 21, 11],
 [18, 13, 21, 11, 9],
 [13, 21, 11, 9, 0],
 [21, 11, 9, 0, 0]]


In [45]:
 [[i2w[token]  for token in tweet_tokens] for tweet_tokens in contextwin(transform_train_tweet[10],context_window)]

[['<padding>', '<padding>', '@sanluispotoyees', '*', '183'],
 ['<padding>', '@sanluispotoyees', '*', '183', '...'],
 ['@sanluispotoyees', '*', '183', '...', '-.-'],
 ['*', '183', '...', '-.-', '<padding>'],
 ['183', '...', '-.-', '<padding>', '<padding>']]

In [46]:
X_train= [ contextwin(tweet,context_window) for tweet in transform_train_tweet]

In [47]:
X_train[:5]

[[[0, 0, 2, 3, 0], [0, 2, 3, 0, 0]],
 [[0, 0, 4, 5, 6], [0, 4, 5, 6, 0], [4, 5, 6, 0, 0]],
 [[0, 0, 7, 8, 9], [0, 7, 8, 9, 0], [7, 8, 9, 0, 0]],
 [[0, 0, 10, 11, 0], [0, 10, 11, 0, 0]],
 [[0, 0, 10, 0, 0]]]

In [48]:
#flatten the list
# we do not need boundaries between tweets as each token now is represented by its context.
X_train=list(chain(*X_train))

In [49]:
X_train=np.array(X_train)

In [50]:
X_train[:5]

array([[0, 0, 2, 3, 0],
       [0, 2, 3, 0, 0],
       [0, 0, 4, 5, 6],
       [0, 4, 5, 6, 0],
       [4, 5, 6, 0, 0]])

In [51]:
Y_train=list(chain(*transform_train_labels))

In [52]:
Y_train=np_utils.to_categorical(np.array(Y_train)) # one hot encoding 

In [53]:
Y_train[:5]

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [54]:
#same for dev data

X_dev=[ contextwin(tweet,context_window) for tweet in transform_dev_tweet]
X_dev=list(chain(*X_dev))

In [55]:
X_dev=np.array(X_dev)

In [56]:
Y_dev=list(chain(*transform_dev_labels))

In [57]:
Y_dev=np_utils.to_categorical(np.array(Y_dev))

In [58]:
#check if train instances and labels are same
assert len(X_train)==len(Y_train)
assert len(X_dev)==len(Y_dev)

###  Building training data for character model

We represent each token by sequence of character in the token. 

In [59]:
X_train_char=list(chain(*tranform_train_tweet_chars))

In [60]:
X_train_char[:5]

[[2, 3, 4, 5, 6, 7, 8],
 [9, 10],
 [11, 12, 13, 8, 7, 14, 15, 6, 16, 17, 6, 3, 7],
 [18],
 [19, 19, 19]]

In [61]:
# creating same length training instances 
# we define sequence length to be the max token length.

X_train_char= sequence.pad_sequences(X_train_char, maxlen=MAX_LEN)

In [62]:
X_train_char=np.array(X_train_char)

In [63]:
X_train_char[:5]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  4,  5,  6,  7,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 11, 12, 13,  8,  7, 14, 15,  6, 16, 17,  6,  3,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 

In [64]:
X_dev_char=list(chain(*tranform_dev_tweet_chars))

In [65]:
X_dev_char=sequence.pad_sequences(X_dev_char, maxlen=MAX_LEN)

## Building Model

<img src="img/model.png" alt="Drawing" style="width: 300px; height: 400px"/>

In [66]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Merge
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.initializers import RandomUniform
from keras.optimizers import Adamax

In [73]:
batch_size=128
nb_epoch=1

In [68]:
# not using pre-trained embedding (randomly initilaized)
# the paper used pre-trained embedding
word_model=Sequential([
    Embedding(output_dim=300, input_dim=n_words,
                    input_length=context_window,mask_zero=True,embeddings_initializer=RandomUniform(seed=1234)),
    Dropout(0.5),
    LSTM(100, return_sequences=False),
    Dropout(0.5)
])
word_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 300)            5296500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 300)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
Total params: 5,456,900
Trainable params: 5,456,900
Non-trainable params: 0
_________________________________________________________________


In [69]:
character_model=Sequential([
    Embedding(n_chars, 50, input_length=MAX_LEN, mask_zero=True,embeddings_initializer=RandomUniform(seed=1234)),
    Dropout(0.5),
    LSTM(100,return_sequences=False),
    Dropout(0.5)
    ])
character_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            16700     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
Total params: 77,100
Trainable params: 77,100
Non-trainable params: 0
_________________________________________________________________


In [70]:
merge_word_character_model = Sequential()
merge_word_character_model.add(Merge([character_model,word_model], mode='concat',concat_axis=1))
merge_word_character_model.add(Dense(output_dim=n_tags, activation='softmax'))
merge_word_character_model.summary()

/Users/suraj/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
/Users/suraj/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1608      
Total params: 5,535,608
Trainable params: 5,535,608
Non-trainable params: 0
_________________________________________________________________


In [71]:
adamax = Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [72]:
merge_word_character_model.compile(loss='categorical_crossentropy', optimizer=adamax, metrics=['accuracy'])

In [74]:
merge_word_character_model.fit([X_train_char,X_train], Y_train,
                batch_size=batch_size,
                epochs=nb_epoch,
                verbose=1,
                shuffle=True,                  
                validation_data=([X_dev_char,X_dev],Y_dev)
                )

Train on 139539 samples, validate on 33276 samples
Epoch 1/1
139539/139539 [==============================] - 415s - loss: 0.4089 - acc: 0.8835 - val_loss: 0.2682 - val_acc: 0.9276

### Getting the merge layer ouput

We can get the output from the merge layer (concatenation of word and character features) and plot them to 2D space using PCA or tsne.

In [78]:
from keras.models import Model

get_merge_layer_output= Model(inputs=[word_model.input,character_model.input], outputs=merge_word_character_model.layers[0].output, name='merge_layer_out')

In [79]:
train_features=get_merge_layer_output.predict([X_train[:5],X_train_char[:5]])

In [80]:
train_features.shape

(5, 200)